# 3D Reconstruction
Now that we have the silhuettes we can now extract the visual hull of the object. Do so we need the camera intrinsic parameters and the extrinsic paramters. Our simulation pipeline already provides us with a json file for that.

In [2]:
import json

import numpy as np

from functools import singledispatchmethod

In [27]:
class ExtrinsicParameters:
    @singledispatchmethod
    def __init__(self, data) -> None:
        raise TypeError(f"Unsupported init type: {type(data)!r}")
    
    @__init__.register
    def _(self, matrix: np.ndarray, position: int) -> None:
        self.matrix = matrix
        self.position = position

    @__init__.register
    def _(self, matrix: np.ndarray) -> None:
        self.matrix = matrix
        self.position = None

    @__init__.register
    def _(self, matrix: list) -> None:
        self.matrix = np.array(matrix)
        self.position = None

    
    @property
    def matrix(self) -> np.ndarray:
        return self._matrix
    
    @matrix.setter
    def matrix(self, value: np.ndarray) -> None:
        if value.shape != (4, 4):
            raise ValueError("Extrinsic matrix must be of shape (4, 4)")
        self._matrix = value


    @property
    def position(self) -> int:
        return self._position
    
    @position.setter
    def position(self, value: int | None) -> None:
        if isinstance(value, None):
            print("⚠️ Warning: No camera position was defined, position is set to None")
            self._position = value
            return
        elif not isinstance(value, int):
            raise TypeError("Position must be an integer")
        self._position = value
    

In [ ]:
class ExtrinsicGroup:
    @singledispatchmethod
    def __init__(self, data) -> None:
        raise TypeError(f"Unsupported init type: {type(data)!r}")
    
    @__init__.register
    def _(self, extrinsics: np.ndarray[ExtrinsicParameters]) -> None:
        self.extrinsics = extrinsics

    @__init__.register
    def _(self, extrinsics: list[ExtrinsicParameters]) -> None:
        self.extrinsics = np.array(extrinsics)

    @property
    def extrinsics(self) -> np.ndarray[ExtrinsicParameters]:
        return self._extrinsics
    
    @extrinsics.setter
    def extrinsics(self, value: np.ndarray[ExtrinsicParameters]) -> None:
        if value.ndim != 1:
            raise ValueError("Extrinsics must be a 1D array of ExtrinsicParameters")
        for ext in value:
            if not isinstance(ext, ExtrinsicParameters):
                raise TypeError("All elements must be of type ExtrinsicParameters")
        self._extrinsics = value

In [24]:
class IntrinsicParameters:
    @singledispatchmethod
    def __init__(self, data) -> None:
        raise TypeError(f"Unsupported init type: {type(data)!r}")
    
    @__init__.register
    def _(self, matrix: np.ndarray) -> None:
        self.matrix = np.round(matrix, 3)

    @__init__.register
    def _(self, matrix: list) -> None:
        self.matrix = np.array(matrix)

    @__init__.register
    def _(self, json_path: str) -> None:
        with open(json_path, 'r') as f:
            data = json.load(f)
        self.matrix = np.array(data['K'])

    
    @property
    def matrix(self) -> np.ndarray:
        return self._matrix
    
    @matrix.setter
    def matrix(self, value: np.ndarray) -> None:
        if value.shape != (3, 3):
            raise ValueError("Intrinsic matrix must be of shape (3, 3)")
        self._matrix = value

In [25]:
IntrinsicParameters("../blender_simulator/simulated_frames/dataset_d30/camera_intrinsics.json").matrix

array([[2.66666667e+03, 0.00000000e+00, 9.60000000e+02],
       [0.00000000e+00, 2.25000000e+03, 5.40000000e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [ ]:
class CamerasData:
    @singledispatchmethod
    def __init__(self, data) -> None:
        raise TypeError(f"Unsupported init type: {type(data)!r}")
    
    @__init__.register
    def _(self, intrinisc: IntrinsicParameters, *extrinsics) -> None:
        self.intrinisc = intrinisc
        self.extrinsics = extrinsics

    @__init__.register
    def _(self, intrinisc_path: str, extrinsics_path: str) -> None:
        self.intrinisc = IntrinsicParameters(intrinisc_path)
        
        with open(extrinsics_path, 'r') as f:
            extrinsics_data = json.load(f)
        extrinsics = []
        for ext in extrinsics_data:
            extrinsics.append(ExtrinisicParameters(ext['matrix'], ext.get('position', None)))
        
        self.intrinisc = intrinisc
        self.extrinsics = extrinsics

In [55]:
camera_data = CamerasData("../blender_simulator/simulated_frames/dataset_d30/transforms_train.json", "../blender_simulator/simulated_frames/dataset_d30/transforms_train.json")

In [ ]:
trf_mt = camera_data.extract_extrinsic_matrices()

In [52]:
len(trf_mt)

12

In [53]:
trf_mt[0]

array([[ 9.99992847e-01,  5.45056282e-05,  3.79700959e-03,
         0.00000000e+00],
       [ 3.79740074e-03, -1.43533014e-02, -9.99889851e-01,
        -1.00000000e+00],
       [-3.63797881e-12,  9.99897003e-01, -1.43534066e-02,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [ ]:
camera_data.extrinsic_matrices

[array([[ 9.99992847e-01,  5.45056282e-05,  3.79700959e-03,
          0.00000000e+00],
        [ 3.79740074e-03, -1.43533014e-02, -9.99889851e-01,
         -1.00000000e+00],
        [-3.63797881e-12,  9.99897003e-01, -1.43534066e-02,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]]),
 array([[ 8.61065686e-01, -7.28784548e-03, -5.08441567e-01,
         -4.99882162e-01],
        [-5.08493781e-01, -1.23409843e-02, -8.60977232e-01,
         -8.65877032e-01],
        [-4.65661287e-10,  9.99897361e-01, -1.43322209e-02,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]]),
 array([[ 0.48193851, -0.01245331, -0.87611651, -0.86587703],
        [-0.87620503, -0.00684969, -0.48188984, -0.49988216],
        [ 0.        ,  0.99989909, -0.01421278,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[-2.57461872e-02, -1.40279029e-02, -9.9957